## Parameters

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import pyarrow.feather as feather

In [2]:
# p_symbol = "BTCUSDT"
# p_symbol = "ETHUSDT"
p_symbol = "SOLUSDT"
p_url = "https://api.binance.com/api/v3/depth"
d_params = {"symbol" :  p_symbol,
            "limit" : 5000}

## Snapshot

#### GET data

In [3]:
d01 = requests.get(p_url,d_params).json()
# d01

In [4]:
# d01

#### TRANSFORM data

In [5]:
def create_df_l2_02(json_str, timestamp, s_sym_u):
    # data_dict = json.loads(json_str)    
    data_dict = json_str
    last_update_id = data_dict["lastUpdateId"]
    bids = pd.DataFrame(data_dict["bids"], columns=["Price", "Quantity"])
    asks = pd.DataFrame(data_dict["asks"], columns=["Price", "Quantity"])
    bids["Side"] = "BID"
    asks["Side"] = "ASK"
    bids["LastUpdateId"] = last_update_id
    asks["LastUpdateId"] = last_update_id

    print("create_df_l2_02: ")
    print(bids.head(2))
    print(asks.head(2))
    print(len(asks))
    print(len(bids))


    # Concatenate bids and asks into a single DataFrame
    df = pd.concat([bids, asks], ignore_index=True)    
    # df["Price"] = df["Price"].astype(float)
    # df["Quantity"] = df["Quantity"].astype(float)
    df["Ticker"] = s_sym_u
    df["Timestamp"] = timestamp
    df = df[["Ticker","Side","Price","Quantity","Timestamp","LastUpdateId"]] 
    print(df.head(2))
    display(len(df))

    return df

In [6]:
current_time = datetime.now()
df_snapshot = create_df_l2_02(d01,current_time,d_params['symbol'],)

create_df_l2_02: 
          Price     Quantity Side  LastUpdateId
0  169.06000000  36.56100000  BID   11388108190
1  169.05000000  84.95100000  BID   11388108190
          Price      Quantity Side  LastUpdateId
0  169.07000000  126.88500000  ASK   11388108190
1  169.08000000  190.75000000  ASK   11388108190
5000
5000
    Ticker Side         Price     Quantity                  Timestamp  \
0  SOLUSDT  BID  169.06000000  36.56100000 2024-05-24 06:59:13.235431   
1  SOLUSDT  BID  169.05000000  84.95100000 2024-05-24 06:59:13.235431   

   LastUpdateId  
0   11388108190  
1   11388108190  


10000

In [7]:
display(df_snapshot.head())
print(df_snapshot.info())

,Ticker,Side,Price,Quantity,Timestamp,LastUpdateId
0,SOLUSDT,BID,169.06000000,36.56100000,2024-05-24 06:59:13.235431,11388108190
1,SOLUSDT,BID,169.05000000,84.95100000,2024-05-24 06:59:13.235431,11388108190
2,SOLUSDT,BID,169.04000000,27.54500000,2024-05-24 06:59:13.235431,11388108190
3,SOLUSDT,BID,169.03000000,250.17900000,2024-05-24 06:59:13.235431,11388108190
4,SOLUSDT,BID,169.02000000,349.54000000,2024-05-24 06:59:13.235431,11388108190


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Ticker        10000 non-null  object        
 1   Side          10000 non-null  object        
 2   Price         10000 non-null  object        
 3   Quantity      10000 non-null  object        
 4   Timestamp     10000 non-null  datetime64[us]
 5   LastUpdateId  10000 non-null  int64         
dtypes: datetime64[us](1), int64(1), object(4)
memory usage: 468.9+ KB
None


In [8]:
p_persist_to = "001_raw"
p_data = "l2s"
s_sym_u = d_params['symbol']
s_gen = "G002_00502"

#### PERSIST data

In [9]:
filename = f"../data/{p_persist_to}/{p_data}_{s_sym_u}_{s_gen}_{current_time.strftime('%Y%m%d_%H%M')}"     
feather.write_feather(df_snapshot, filename+".feather")            
df_snapshot.to_csv(filename+".csv", index=False)
print("export: "+str(filename)+" | length: "+str(len(df_snapshot)))  

export: ../data/001_raw/l2s_SOLUSDT_G002_00502_20240524_0659 | length: 10000


In [10]:
# filename